## i. Perkenalan
`Marchelly Gideon Tambunan`\

# Working Area

### Problem Statement

Sebelum Membuat laporan evaluasi data, ada baiknya kita membuat success criteria dengan SMART.

S: Meningkatkan pendapatan pada kuartal 4

M: Meningkatnya jumlah user dan transaksi

A: Meningkatkan penjualan atau transaksi

R: Mencari Investor dan Memasarkan Produk

T: Kuartal 4

Sebelum masuk pada poin Penjabaran, import library yang diperlukan dan connect google Cloud pada google Colab

In [ ]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "marchelly-gideon" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


## Poin Penjabaran

1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [ ]:
jumlah_transaksi_complete = client.query('''
  SELECT 
    EXTRACT(YEAR FROM created_at) AS year, 
    EXTRACT(MONTH FROM created_at) AS month, 
    EXTRACT(QUARTER FROM created_at) AS quarter,
    COUNT(order_id) as complete
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE status='Complete' AND created_at >= "2022-01-01" AND created_at <= "2022-09-30"
GROUP BY year,month,quarter
ORDER BY year,month ASC
''').to_dataframe()

jumlah_transaksi_complete

,year,month,quarter,complete
0,2022,1,1,820
1,2022,2,1,800
2,2022,3,1,946
3,2022,4,2,927
4,2022,5,2,1022
5,2022,6,2,1086
6,2022,7,3,1308
7,2022,8,3,1317
8,2022,9,3,1418


Dari hasil diatas, dapat dilihat bahwa transaksi pada Q3 adalah 3 bulan terbanyak dibandingkan bulan yang lainnya

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [ ]:
TotaL_penjualan_berhasil = client.query('''
SELECT 
    EXTRACT(YEAR FROM created_at) AS year, 
    EXTRACT(MONTH FROM created_at) AS month, 
    EXTRACT(QUARTER FROM created_at) AS quarter,
    FLOOR(SUM(sale_price)) AS total_penjualan
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status='Complete' AND created_at >= "2022-01-01" AND created_at <= "2022-09-30"
GROUP BY year,month,quarter
ORDER BY year,month ASC 
''').to_dataframe()

TotaL_penjualan_berhasil

,year,month,quarter,total_penjualan
0,2022,1,1,73970.0
1,2022,2,1,67969.0
2,2022,3,1,81795.0
3,2022,4,2,82989.0
4,2022,5,2,89865.0
5,2022,6,2,90493.0
6,2022,7,3,109779.0
7,2022,8,3,118612.0
8,2022,9,3,122763.0


Karena jumlah transaksi berhasil terbanyak pada Q3, jadi total penjualan berhasil terbanyak pun ada pada Q3

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [ ]:
TotaL_user_berhasil = client.query('''
SELECT 
    EXTRACT(YEAR FROM created_at) AS year, 
    EXTRACT(MONTH FROM created_at) AS month, 
    EXTRACT(QUARTER FROM created_at) AS quarter,
    COUNT(DISTINCT user_id) AS total_user
FROM `bigquery-public-data.thelook_ecommerce.order_items`
WHERE status='Complete' AND created_at >= "2022-01-01" AND created_at <= "2022-09-30"
GROUP BY year,month,quarter
ORDER BY year,month ASC 
''').to_dataframe()

TotaL_user_berhasil

,year,month,quarter,total_user
0,2022,1,1,832
1,2022,2,1,810
2,2022,3,1,954
3,2022,4,2,932
4,2022,5,2,1028
5,2022,6,2,1101
6,2022,7,3,1323
7,2022,8,3,1332
8,2022,9,3,1412


seperti pada poin 1 dan 2, user yang melakukan trannsaksi terbanyak ada pada Q3

4. Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [ ]:
produk_paling_laku = client.query('''
SELECT *
FROM
(SELECT *,
     DENSE_RANK() OVER(PARTITION BY month ORDER BY category_sold DESC) AS category_rank
FROM
(SELECT EXTRACT(YEAR FROM order_items.created_at) AS year,
        EXTRACT(QUARTER FROM order_items.created_at) AS quarter,
        EXTRACT(MONTH FROM order_items.created_at) AS month,
        products.category, 
        COUNT(order_items.order_id) AS category_sold,
FROM bigquery-public-data.thelook_ecommerce.order_items AS order_items 
JOIN bigquery-public-data.thelook_ecommerce.products AS products
ON order_items.product_id = products.id
WHERE order_items.status = 'Complete' 
  AND order_items.created_at >= '2022-01-01' 
  AND order_items.created_at <= '2022-09-30'
GROUP BY month, quarter, year, products.category
ORDER BY month, category_sold desc)
ORDER BY month, category_sold desc, category_rank)
WHERE category_rank in (1)
''').to_dataframe()

produk_paling_laku

,year,quarter,month,category,category_sold,category_rank
0,2022,1,1,Jeans,91,1
1,2022,1,2,Intimates,93,1
2,2022,1,2,Tops & Tees,93,1
3,2022,1,3,Jeans,106,1
4,2022,2,4,Jeans,102,1
5,2022,2,5,Fashion Hoodies & Sweatshirts,115,1
6,2022,2,6,Sweaters,109,1
7,2022,3,7,Fashion Hoodies & Sweatshirts,139,1
8,2022,3,8,Jeans,132,1
9,2022,3,9,Intimates,155,1


Pada hasil diatas, category yang mendominasi penjualan adalah Intimates

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [ ]:
produk_paling_untung = client.query('''
SELECT *
FROM
(SELECT *,
     DENSE_RANK() OVER(PARTITION BY month ORDER BY revenue desc) AS category_rank
FROM
(SELECT EXTRACT(YEAR FROM order_items.created_at) AS year,
        EXTRACT(QUARTER FROM order_items.created_at) AS quarter,
        EXTRACT(MONTH FROM order_items.created_at) AS month,
        products.category, COUNT(order_items.order_id) AS category_sold,
        FLOOR(SUM(sale_price)) AS revenue,
FROM bigquery-public-data.thelook_ecommerce.order_items AS order_items 
JOIN bigquery-public-data.thelook_ecommerce.products AS products
ON order_items.product_id = products.id
WHERE order_items.status = 'Complete' 
  AND order_items.created_at >= '2022-01-01' 
  AND order_items.created_at <= '2022-09-30'
GROUP BY month, quarter, year, products.category
ORDER BY month, revenue desc)
ORDER BY month, revenue desc, category_rank)
WHERE category_rank in (1)
''').to_dataframe()

produk_paling_untung

,year,quarter,month,category,category_sold,revenue,category_rank
0,2022,1,1,Jeans,91,9886.0,1
1,2022,1,2,Outerwear & Coats,59,8316.0,1
2,2022,1,3,Jeans,106,11589.0,1
3,2022,2,4,Outerwear & Coats,80,13457.0,1
4,2022,2,5,Outerwear & Coats,86,13022.0,1
5,2022,2,6,Jeans,98,10712.0,1
6,2022,3,7,Jeans,122,12933.0,1
7,2022,3,8,Outerwear & Coats,109,14699.0,1
8,2022,3,9,Outerwear & Coats,123,19964.0,1


pada poin ini menunjukan ternyata produuk yang paling banyak mendapatkan keuntungan adalah produk Outerwear & Coats

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? (Hint: Kamu bisa menggunakan korelasi)

In [ ]:
korelasi = jumlah_transaksi_complete.copy()
korelasi['total_transaksi'] = TotaL_penjualan_berhasil['total_penjualan']
korelasi['total_user'] = TotaL_user_berhasil['total_user']
korelasi.drop(columns=['year','month','quarter'], inplace=True)

korelasi.corr()

,complete,total_transaksi,total_user
complete,1.000000,0.989566,0.999565
total_transaksi,0.989566,1.000000,0.988433
total_user,0.999565,0.988433,1.000000


Yang paling berkaitan dengan user adalah complete karena setiap user pasti harus berhasil melakukan transaksi guna mencapai keuntungan yang maksimal

## Poin Analisis

Setelah melakukan beberapa eksplorasi, analisis data. Kesimpulan yang dapat diambil adalah keuntungan tidak bergantung pada transaksi. Artinya, pendapatan yang didapat dari sebuah produk bergantung dari biaya produksi dan harga jual produk tersebut.

Dilihat dari rentang waktu Q1 sampai Q3, Transaksi terbanyak berada pada Q3. Dapat menarik kesimpulan bahwa pada waktu tersebut user sedang mendapat banyak melakukan transaksi. apabila kita ingin menyeimbangkan dengan waktu sebelumnya, berarti kita perlu melakukan promo pada Q1 dan Q2 agar Transaksi pada waktu tersebut dapat meningkat.

## Poin Pertanyaan

Pertanyaan:
1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!
2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?
3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

Jawaban:
1. ya, karena user pasti menginginkan promo dan produk yang menarik
2. Menurut saya, tidak masuk akal. karena, dilihat dari data pendapatan yang didapatkan setiap bulannya masih sangat jauh dari keinginan CEO
3. Informasi yang bisa didapatkan yaitu pada kuartal 4 yang berarti akan memasuki event natal dan tahun baru yang sudah pasti akan meningkatkan pendapatan. selain itu, user akan mendapatkan THR dan libur yang cukup banyak dimana akan memerlukan produk baru yang akan digunakan sehari-hari seperti pakaian baru. oleh karena itu, inilah saatnya untuk investor baru untuk datang dan bertumbuh bersama di The Look E-commerce!!